### SageMaker  Endpoint 部署ChatYuan
  
[ChatYuan](https://huggingface.co/ClueAI/ChatYuan-large-v1): 元语功能型对话大模型 , 这个模型可以用于问答、结合上下文做对话、做各种生成任务，包括创意性写作，也能回答一些像法律、新冠等领域问题。它基于PromptCLUE-large结合数亿条功能对话多轮对话数据进一步训练得到。

PromptCLUE-large在1000亿token中文语料上预训练，累计学习1.5万亿中文token，并且在数百种任务上进行Prompt任务式训练。针对理解类任务，如分类、情感分析、抽取等，可以自定义标签体系；针对多种生成任务，可以进行采样自由生成。

#### 准备
1. 升级boto3, sagemaker python sdk  
2. 准备inference.py, requirements.txt

In [1]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 34.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.71
    Uninstalling botocore-1.29.71:
      Successfully uninstalled botocore-1.29.71
  Attempting uninstall: boto3
    Found existing installation: boto3 1.26.71
    Uninstalling boto3-1.26.71:
      Successfully uninstalled boto3-1.26.71
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.71 requires botocore==1.29.71, but you have botocore 1.29.84 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 674.4/674.4 kB 24.0 MB/s 

In [2]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

arn:aws:iam::710299592439:role/service-role/AmazonSageMaker-ExecutionRole-20200113T084273
sagemaker-us-east-1-710299592439


In [3]:
!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'chatyuan')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'chatyuan')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

dummy
upload: ./model.tar.gz to s3://sagemaker-us-east-1-710299592439/chatyuan/assets/model.tar.gz


### 设置模型推理参数
1. model_name: Huggingface diffusers models (not support single check point format)
2. model_args: diffuser StableDiffusionPipeline init arguments

In [4]:
model_name = None
entry_point = 'inference.py'
framework_version = '1.12'
py_version = 'py38'
bucket='m.flowq.io'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

In [5]:
from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    name = model_name,
    model_data = model_data,
    entry_point = 'inference.py',
    source_dir = './code',
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

### 部署 SageMaker Endpoint
部署推理服务

In [6]:
endpoint_name = None
instance_type = 'ml.g4dn.xlarge'
instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

---------!

### ChatYuan 测试


In [7]:
inputs= {
    "ask": "费德勒网球水平怎么样? "

}
response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "帮我写一个如何给小学二年级学生讲述AIGC的文章"

}
response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "如何提升云计算解决方案架构师的ML能力，写一个600字建议"

}
response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "请给出中国2024年自动驾驶市场成熟度和最具影响力的汽车品牌的预测"

}
response = predictor.predict(inputs)
print(response["answer"])

费德勒的球技怎么样? 费德勒,世界排名第一的网球选手,他也是当今网球界最顶尖的球员之一,他拥有着出色的技术,他的实力也是非常的强,他的球技也是非常的好,他的比赛也是非常的精彩,他的世界排名也是非常高的,他曾经获得过很多的冠军,他的成绩也是非常的优秀,他现在也是世界排名第一,他还是世界排名第一。 费天王的球风也是非常的独特,他打球的风格也是非常多变的,他的技术也是非常的强大,他的打球的风格也特别的多变,他打法也是非常独特,他的风格也是比较的多变,他的打法是比较的简单,他的击球也是非常轻松的,费德勒打法非常的多变。 他打的球也是非常具有观赏性的,他的发球也是很具有观赏性,他的进攻也是非常犀利的,他在发球方面也是非常有优势的,在发球的方面也是比较有优势,他的得分也是非常多的,他是世界排名第一了,他的对手也是非常强的,他们都是世界排名前十的球员,他们也是非常厉害的,他们的球技都是非常的厉害的。 他的球风
，题目是AIGC，要求写一篇关于AIG C的文章。AIG 是人工智能(AI)的缩写，是计算机视觉(ComputerVision)的简称，是人工智能的简称。AI GC 是计算机图形学(ComputingGraphics)的英文缩写，是指计算机视觉技术(CompositeGraphic Technology)。AI 是计算机科学的一个分支，是计算机科学中一个重要的分支。AI是计算机图像处理(ComparativeGraphical Analysis)的简写，是图像处理的简称;AI 也是计算机视觉的简称(Composed Vision Teaching)，是视觉识别(Visual Answer)的俗称。AI的英文全称是Automatically Graphed Animation，中文翻译为计算机视觉。AI是一种计算机视觉，它通过计算机视觉算法来识别物体，并进行识别。AI算法是计算机图像识别的一种方法，它能够识别物体的轮廓、颜色、形状、大小、位置、大小和形状，以及物体的尺寸、形状和位置。AI
？首先，我们需要明确一点，云计算架构师需要具备的ML能力，是需要通过实践来提升的。那么，如何提升云计算架构工程师的ML水平呢？首先需要了解，云计算的架构师，需要具备哪些ML能力？其次，如何提高云计算架构的ML技术？最后，如何解决云计算架构中遇到的一些问题？最后需要说明的是，云计算技术的发展，

In [8]:
#我们来查看一下刚部署的这个chatyuan模型对应的SageMaker inference endpoint
your_chatyuan_endpoint_name = predictor.endpoint_name
your_chatyuan_endpoint_name

'pytorch-inference-2023-03-04-15-19-29-185'

利用已经在SageMaker real time inference endpoint部署的ChatYuan LLM模型来模拟单轮对话和多轮对话。

下面的代码建议在SageMaker Notebook上来运行。

In [9]:
import json
import boto3

client = boto3.client('runtime.sagemaker')

def query_endpoint_with_json_payload(encoded_json):
    response = client.invoke_endpoint(EndpointName=your_chatyuan_endpoint_name, ContentType='application/json', Body=encoded_json)
    return response

def parse_response_texts(query_response):
    model_predictions = json.loads(query_response['Body'].read())
    generated_text = model_predictions["answer"]
    return generated_text


先简单测试一下chatyuan针对单个问题的回答

In [10]:
payload = {"ask": "信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案："}
query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
generated_texts = parse_response_texts(query_response)
print(generated_texts)

国家名：阿根廷
人名：梅西


单轮对话：每个问题/query都是独立的，问题之间没有关联性。

In [11]:
#1.首先需要一个简单的开场拍。
print("用户：你好！\n小元：您好!我是元语AI。我可以回答您的问题、写文章、写作业、翻译，对于一些法律等领域的问题我也可以给你提供信息。")

#2.在同一个session中持续对话，但是每次对话之间没有什么关联。
while True:
    command = input("用户：")
    if command == 'quit':
        break;
    
    #print(command)
    payload = {"ask": "\n用户："+ command + "\n"}
    #print(payload["ask"])
    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
    generated_texts = "小元：" + parse_response_texts(query_response)
    print(generated_texts)

用户：你好！
小元：您好!我是元语AI。我可以回答您的问题、写文章、写作业、翻译，对于一些法律等领域的问题我也可以给你提供信息。


用户： AWS Redshift是什么？


小元：redshift 是 AWS 的一个开源的 API，用于提供 Web 服务的 Web Services API。redshify 是 red shift， red-shift。red Shift 是一个 Web servicing API ，用于在 WebServ 服务中提供 web 服务。red-Shift的 web Servision APIs 可以提供 https 和 https-https 两种方式。redShift-web SERVICE API是 https://redshife.redshiffy.com/redshifile-web-servics-services-server-servicing-servers-serving-webservi


用户： 你和微软小秘谁更厉害


小元：微软小秘厉害


...

用户： 我在问你问题


小元：什么问题


用户： 小元的中文能力不太行，怎么提高？


小元：小的中文能力不太行，怎么提高？


用户： quit


多轮对话模拟：我们这里来测试一下chatyuan的多轮对话能力。

In [13]:
#1.首先需要开场拍来引导chatyuan，其实就是给它一个上下文来启动所谓的对话session。
payload = {"ask": "用户：你好！\n小元：您好!我是元语AI。我可以回答您的问题、写文章、写作业、翻译，对于一些法律等领域的问题我也可以给你提供信息。"}
print(payload["ask"])
generated_texts = ""

#在这里简单模拟多轮对话时，发送给SageMaker endpoint的payload会越来越大，这里对payload大致做一个限制。
session_len = 10 * 1024 * 1024 

#2.在同一个session中持续对话，为了有多轮对话的效果，把每一轮的信息(问题-回答对)都带上来告诉chatyuan之前的上下文。
while True:
    command = input("用户：")
    if command == 'quit':
        break;
    
    #print(command)
    whole_context = payload["ask"] + generated_texts + "\n用户："+ command + "\n"
    payload = {"ask": whole_context}
    if len(whole_context) > session_len:
        print("上下文信息太多了，当前对话session要退出了！")
        break;
    #print(payload["ask"])
    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
    generated_texts = "小元：" + parse_response_texts(query_response)
    print(generated_texts)

用户：你好！
小元：您好!我是元语AI。我可以回答您的问题、写文章、写作业、翻译，对于一些法律等领域的问题我也可以给你提供信息。


用户： quit


### 删除SageMaker  Endpoint
删除推理服务

In [14]:
predictor.delete_endpoint()